# Image Collection Collage Generator

Questo notebook crea collage di immagini filtrate per round, classe e nodo da una cartella di immagini.

Pattern dei nomi file: `round_{round}_node_{node}_img_{class}_{index}[_variant].png`

**Varianti:**
- **test**: immagini senza suffisso (generate durante il test)
- **train**: immagini con suffisso `_train` (generate durante il training)
- **da_embedding**: immagini con `from_textembs` (generate da text embeddings)

In [1]:
import os
import re
from pathlib import Path
from typing import List, Optional, Tuple
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
def parse_image_filename(filename: str) -> dict:
    """
    Parsa il nome del file per estrarre metadati.
    
    Pattern: round_{round}_node_{node}_img_{class}_{index}[_variant].png
    
    Varianti:
    - 'test': immagini senza suffisso (default)
    - 'train': immagini con suffisso _train
    - 'da_embedding': immagini con from_textembs
    
    Returns:
        dict con chiavi: round, node, class_name, index, variant, full_path
    """
    pattern = r'round_(\d+)_node_(\d+)_img_([a-zA-Z_]+)_(\d+|from_textembs)(?:_([a-zA-Z_]+))?\.png'
    match = re.match(pattern, filename)
    
    if not match:
        return None
    
    round_num = int(match.group(1))
    node_num = int(match.group(2))
    class_name = match.group(3)
    index = match.group(4)
    suffix = match.group(5)
    
    # Determina la variante
    if index == 'from_textembs':
        variant = 'da_embedding'
        index = 'from_textembs'
    elif suffix == 'train':
        variant = 'train'
    else:
        variant = 'test'
    
    return {
        'round': round_num,
        'node': node_num,
        'class_name': class_name,
        'index': index,
        'variant': variant,
        'filename': filename
    }

In [3]:
def filter_images(image_dir: str, 
                  rounds: Optional[List[int]] = None,
                  nodes: Optional[List[int]] = None,
                  classes: Optional[List[str]] = None,
                  variants: Optional[List[str]] = None,
                  max_images: Optional[int] = None) -> List[dict]:
    """
    Filtra le immagini in base ai criteri specificati.
    
    Args:
        image_dir: Percorso alla cartella contenente le immagini
        rounds: Lista di round da includere (None = tutti)
        nodes: Lista di nodi da includere (None = tutti)
        classes: Lista di classi da includere (None = tutte)
        variants: Lista di varianti da includere (es. ['test', 'train', 'da_embedding'])
        max_images: Numero massimo di immagini da includere
        
    Returns:
        Lista di dizionari con metadati delle immagini filtrate
    """
    image_dir = Path(image_dir)
    filtered_images = []
    
    for img_file in sorted(image_dir.glob('*.png')):
        metadata = parse_image_filename(img_file.name)
        
        if metadata is None:
            continue
            
        # Applica filtri
        if rounds is not None and metadata['round'] not in rounds:
            continue
        if nodes is not None and metadata['node'] not in nodes:
            continue
        if classes is not None and metadata['class_name'] not in classes:
            continue
        if variants is not None and metadata['variant'] not in variants:
            continue
            
        metadata['full_path'] = str(img_file)
        filtered_images.append(metadata)
        
        if max_images and len(filtered_images) >= max_images:
            break
    
    return filtered_images

In [4]:
def create_collage(images_metadata: List[dict], 
                   cols: int = 4,
                   img_size: Tuple[int, int] = (256, 256),
                   show_labels: bool = True,
                   figsize: Optional[Tuple[int, int]] = None,
                   save_path: Optional[str] = None) -> plt.Figure:
    """
    Crea un collage di immagini.
    
    Args:
        images_metadata: Lista di dizionari con metadati delle immagini
        cols: Numero di colonne nel collage
        img_size: Dimensione di ridimensionamento per ogni immagine (width, height)
        show_labels: Se True, mostra le etichette sotto ogni immagine
        figsize: Dimensione della figura (width, height). Se None, calcolata automaticamente
        save_path: Se specificato, salva il collage in questo percorso
        
    Returns:
        Figure matplotlib
    """
    if not images_metadata:
        print("Nessuna immagine da visualizzare")
        return None
    
    n_images = len(images_metadata)
    rows = (n_images + cols - 1) // cols  # Ceil division
    
    # Calcola dimensione figura se non specificata
    if figsize is None:
        fig_width = cols * 3
        fig_height = rows * 3.5 if show_labels else rows * 3
        figsize = (fig_width, fig_height)
    
    fig, axes = plt.subplots(rows, cols, figsize=figsize)
    
    # Assicurati che axes sia sempre un array 2D
    if rows == 1 and cols == 1:
        axes = np.array([[axes]])
    elif rows == 1:
        axes = axes.reshape(1, -1)
    elif cols == 1:
        axes = axes.reshape(-1, 1)
    
    for idx, metadata in enumerate(images_metadata):
        row = idx // cols
        col = idx % cols
        ax = axes[row, col]
        
        # Carica e visualizza immagine
        try:
            img = Image.open(metadata['full_path'])
            img = img.resize(img_size, Image.Resampling.LANCZOS)
            ax.imshow(img)
            
            if show_labels:
                label = f"R{metadata['round']} N{metadata['node']}\n{metadata['class_name']}\n{metadata['variant']}"
                ax.set_title(label, fontsize=8)
        except Exception as e:
            ax.text(0.5, 0.5, f'Error: {str(e)}', 
                   ha='center', va='center', fontsize=8)
        
        ax.axis('off')
    
    # Rimuovi subplot vuoti
    for idx in range(n_images, rows * cols):
        row = idx // cols
        col = idx % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Collage salvato in: {save_path}")
    
    return fig

In [5]:
def get_available_metadata(image_dir: str) -> dict:
    """
    Analizza la cartella e restituisce tutti i valori univoci per round, nodi, classi e varianti.
    
    Args:
        image_dir: Percorso alla cartella contenente le immagini
        
    Returns:
        Dizionario con set di valori univoci
    """
    image_dir = Path(image_dir)
    
    rounds = set()
    nodes = set()
    classes = set()
    variants = set()
    
    for img_file in image_dir.glob('*.png'):
        metadata = parse_image_filename(img_file.name)
        if metadata:
            rounds.add(metadata['round'])
            nodes.add(metadata['node'])
            classes.add(metadata['class_name'])
            variants.add(metadata['variant'])
    
    return {
        'rounds': sorted(rounds),
        'nodes': sorted(nodes),
        'classes': sorted(classes),
        'variants': sorted(variants)
    }

## Interfaccia Interattiva

Usa i widget qui sotto per selezionare la cartella, filtrare le immagini e creare il collage in modo interattivo.

In [7]:
# Classe per gestire l'interfaccia interattiva
class InteractiveCollageGenerator:
    def __init__(self, default_dir="/home/lpala/fedgfe/esc50-6n-global-train"):
        # Widget per la cartella
        self.folder_widget = widgets.Text(
            value=default_dir,
            description='Cartella:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='80%')
        )
        
        # Pulsante per analizzare la cartella
        self.analyze_button = widgets.Button(
            description='Analizza Cartella',
            button_style='info',
            icon='search'
        )
        
        # Widget per dimensione immagini
        self.img_width_widget = widgets.IntSlider(
            value=256,
            min=64,
            max=1024,
            step=64,
            description='Larghezza img:',
            style={'description_width': '120px'}
        )
        
        self.img_height_widget = widgets.IntSlider(
            value=256,
            min=64,
            max=1024,
            step=64,
            description='Altezza img:',
            style={'description_width': '120px'}
        )
        
        # Widget per numero di colonne
        self.cols_widget = widgets.IntSlider(
            value=4,
            min=1,
            max=10,
            step=1,
            description='Colonne:',
            style={'description_width': '120px'}
        )
        
        # Widget per filtri (verranno popolati dopo l'analisi)
        self.rounds_widget = widgets.SelectMultiple(
            options=[],
            description='Rounds:',
            style={'description_width': '120px'},
            layout=widgets.Layout(height='100px', width='180px')
        )
        
        self.nodes_widget = widgets.SelectMultiple(
            options=[],
            description='Nodes:',
            style={'description_width': '120px'},
            layout=widgets.Layout(height='100px', width='180px')
        )
        
        self.classes_widget = widgets.SelectMultiple(
            options=[],
            description='Classi:',
            style={'description_width': '120px'},
            layout=widgets.Layout(height='150px', width='200px')
        )
        
        self.variants_widget = widgets.SelectMultiple(
            options=[],
            description='Varianti:',
            style={'description_width': '120px'},
            layout=widgets.Layout(height='100px', width='180px')
        )
        
        # Pulsanti per deselezionare tutto
        self.clear_rounds_btn = widgets.Button(description='Deseleziona', button_style='warning', layout=widgets.Layout(width='100px'))
        self.clear_nodes_btn = widgets.Button(description='Deseleziona', button_style='warning', layout=widgets.Layout(width='100px'))
        self.clear_classes_btn = widgets.Button(description='Deseleziona', button_style='warning', layout=widgets.Layout(width='100px'))
        self.clear_variants_btn = widgets.Button(description='Deseleziona', button_style='warning', layout=widgets.Layout(width='100px'))
        
        # Widget per opzioni aggiuntive
        self.max_images_widget = widgets.IntText(
            value=100,
            description='Max immagini:',
            style={'description_width': '120px'}
        )
        
        self.show_labels_widget = widgets.Checkbox(
            value=True,
            description='Mostra etichette'
        )
        
        self.save_path_widget = widgets.Text(
            value='collage_output.png',
            description='Nome file:',
            style={'description_width': '120px'},
            layout=widgets.Layout(width='50%')
        )
        
        # Pulsante per generare il collage
        self.generate_button = widgets.Button(
            description='Genera Collage',
            button_style='success',
            icon='image',
            disabled=True
        )
        
        # Output area
        self.output = widgets.Output()
        
        # Info sulla cartella analizzata
        self.info_output = widgets.Output()
        
        # Collega gli eventi
        self.analyze_button.on_click(self.analyze_folder)
        self.generate_button.on_click(self.generate_collage)
        self.clear_rounds_btn.on_click(lambda b: setattr(self.rounds_widget, 'value', ()))
        self.clear_nodes_btn.on_click(lambda b: setattr(self.nodes_widget, 'value', ()))
        self.clear_classes_btn.on_click(lambda b: setattr(self.classes_widget, 'value', ()))
        self.clear_variants_btn.on_click(lambda b: setattr(self.variants_widget, 'value', ()))
        
        # Metadati disponibili
        self.available_metadata = None
        
    def analyze_folder(self, b):
        """Analizza la cartella e aggiorna i widget dei filtri."""
        with self.info_output:
            clear_output()
            folder_path = self.folder_widget.value
            
            if not os.path.exists(folder_path):
                print(f"❌ Errore: La cartella '{folder_path}' non esiste")
                self.generate_button.disabled = True
                return
            
            try:
                print(f"🔍 Analisi della cartella: {folder_path}")
                self.available_metadata = get_available_metadata(folder_path)
                
                # Aggiorna i widget
                self.rounds_widget.options = [f"Round {r}" for r in self.available_metadata['rounds']]
                self.nodes_widget.options = [f"Node {n}" for n in self.available_metadata['nodes']]
                self.classes_widget.options = sorted(self.available_metadata['classes'])
                self.variants_widget.options = sorted(self.available_metadata['variants'])
                
                # Deseleziona tutto
                self.rounds_widget.value = ()
                self.nodes_widget.value = ()
                self.classes_widget.value = ()
                self.variants_widget.value = ()
                
                # Abilita il pulsante di generazione
                self.generate_button.disabled = False
                
                print(f"✅ Analisi completata!")
                print(f"   - Rounds: {self.available_metadata['rounds']}")
                print(f"   - Nodes: {self.available_metadata['nodes']}")
                print(f"   - Classi ({len(self.available_metadata['classes'])}): {', '.join(sorted(self.available_metadata['classes']))}")
                print(f"   - Varianti: {', '.join(sorted(self.available_metadata['variants']))}")
                print(f"\n💡 IMPORTANTE: Lascia i filtri VUOTI per selezionare TUTTO")
                print(f"   Usa Ctrl/Cmd + clic per selezionare più elementi")
                print(f"   Clicca 'Deseleziona' per svuotare un filtro")
                
            except Exception as e:
                print(f"❌ Errore durante l'analisi: {str(e)}")
                self.generate_button.disabled = True
    
    def generate_collage(self, b):
        """Genera il collage in base ai filtri selezionati."""
        with self.output:
            clear_output(wait=True)
            
            try:
                # Estrai i valori dai widget
                folder_path = self.folder_widget.value
                
                # Converti le selezioni in liste (se vuoto = None = tutti)
                rounds = [int(r.split()[1]) for r in self.rounds_widget.value] if len(self.rounds_widget.value) > 0 else None
                nodes = [int(n.split()[1]) for n in self.nodes_widget.value] if len(self.nodes_widget.value) > 0 else None
                classes = list(self.classes_widget.value) if len(self.classes_widget.value) > 0 else None
                variants = list(self.variants_widget.value) if len(self.variants_widget.value) > 0 else None
                
                # Mostra filtri applicati
                print("🔍 Filtri applicati:")
                print(f"   - Rounds: {rounds if rounds else 'TUTTI'}")
                print(f"   - Nodes: {nodes if nodes else 'TUTTI'}")
                print(f"   - Classi: {classes if classes else 'TUTTE'}")
                print(f"   - Varianti: {variants if variants else 'TUTTE'}")
                print(f"   - Max immagini: {self.max_images_widget.value}")
                print()
                
                # Filtra le immagini
                print("🔍 Filtraggio immagini...")
                filtered = filter_images(
                    folder_path,
                    rounds=rounds,
                    nodes=nodes,
                    classes=classes,
                    variants=variants,
                    max_images=self.max_images_widget.value
                )
                
                if not filtered:
                    print("⚠️ Nessuna immagine trovata con i filtri selezionati")
                    print("\n💡 Suggerimento: Prova a deselezionare alcuni filtri o aumenta 'Max immagini'")
                    return
                
                print(f"✅ Trovate {len(filtered)} immagini")
                
                # Mostra distribuzione
                from collections import Counter
                nodes_count = Counter(img['node'] for img in filtered)
                print(f"\nDistribuzione per nodo: {dict(nodes_count)}")
                
                # Crea il collage
                print("\n🎨 Creazione del collage...")
                img_size = (self.img_width_widget.value, self.img_height_widget.value)
                
                fig = create_collage(
                    filtered,
                    cols=self.cols_widget.value,
                    img_size=img_size,
                    show_labels=self.show_labels_widget.value,
                    save_path=self.save_path_widget.value if self.save_path_widget.value else None
                )
                
                if fig:
                    plt.show()
                    print(f"\n✅ Collage generato con successo!")
                    
            except Exception as e:
                print(f"❌ Errore durante la generazione: {str(e)}")
                import traceback
                traceback.print_exc()
    
    def display(self):
        """Mostra l'interfaccia."""
        # Sezione 1: Selezione cartella
        folder_section = widgets.VBox([
            widgets.HTML("<h3>📁 Selezione Cartella</h3>"),
            self.folder_widget,
            self.analyze_button,
            self.info_output
        ])
        
        # Sezione 2: Filtri con pulsanti di deseleziona
        filters_section = widgets.VBox([
            widgets.HTML("<h3>🔍 Filtri</h3>"),
            widgets.HTML("<b style='color: red;'>⚠️ LASCIA VUOTO per selezionare TUTTO</b><br><i>Usa Ctrl/Cmd + clic per selezioni multiple</i>"),
            widgets.HBox([
                widgets.VBox([self.rounds_widget, self.clear_rounds_btn]),
                widgets.VBox([self.nodes_widget, self.clear_nodes_btn]),
                widgets.VBox([self.classes_widget, self.clear_classes_btn]),
                widgets.VBox([self.variants_widget, self.clear_variants_btn])
            ])
        ])
        
        # Sezione 3: Impostazioni collage
        collage_settings = widgets.VBox([
            widgets.HTML("<h3>⚙️ Impostazioni Collage</h3>"),
            widgets.HBox([
                widgets.VBox([
                    self.img_width_widget,
                    self.img_height_widget,
                    self.cols_widget
                ]),
                widgets.VBox([
                    self.max_images_widget,
                    self.show_labels_widget,
                    self.save_path_widget
                ])
            ])
        ])
        
        # Sezione 4: Generazione
        generate_section = widgets.VBox([
            widgets.HTML("<h3>🎨 Genera Collage</h3>"),
            self.generate_button,
            self.output
        ])
        
        # Layout completo
        full_interface = widgets.VBox([
            widgets.HTML("<h2>🖼️ Generatore Collage Interattivo</h2>"),
            folder_section,
            widgets.HTML("<hr>"),
            filters_section,
            widgets.HTML("<hr>"),
            collage_settings,
            widgets.HTML("<hr>"),
            generate_section
        ])
        
        display(full_interface)

# Crea e mostra l'interfaccia
generator = InteractiveCollageGenerator()
generator.display()

## Esempio di utilizzo

### 1. Analizza la cartella per vedere cosa contiene

In [1]:
# Specifica la cartella contenente le immagini
IMAGE_DIR = "/home/lpala/fedgfe/esc50-6n-global-train"

# Ottieni metadati disponibili
metadata = get_available_metadata(IMAGE_DIR)

print("=== Metadati disponibili ===")
print(f"\nRounds: {metadata['rounds']}")
print(f"\nNodes: {metadata['nodes']}")
print(f"\nClasses ({len(metadata['classes'])}): {metadata['classes']}")
print(f"\nVariants: {metadata['variants']}")

NameError: name 'get_available_metadata' is not defined

### 2. Esempio: Collage di una classe specifica per un nodo

In [ ]:
# Filtra immagini: classe "airplane" del nodo 0
filtered = filter_images(
    IMAGE_DIR,
    nodes=[0],
    classes=['airplane'],
    max_images=20
)

print(f"Trovate {len(filtered)} immagini")

# Crea collage
fig = create_collage(
    filtered,
    cols=5,
    show_labels=True,
    save_path="collage_airplane_node0.png"
)
plt.show()

### 3. Esempio: Collage di round specifici

In [ ]:
# Filtra immagini: round 1 e 10
filtered = filter_images(
    IMAGE_DIR,
    rounds=[1, 10],
    variants=['test'],  # Solo immagini test (senza suffisso)
    max_images=30
)

print(f"Trovate {len(filtered)} immagini")

# Crea collage
fig = create_collage(
    filtered,
    cols=6,
    show_labels=True,
    save_path="collage_rounds_1_10.png"
)
plt.show()

### 4. Esempio: Collage di tutte le immagini di un nodo specifico in un round

In [ ]:
# Filtra immagini: nodo 1, round 1
filtered = filter_images(
    IMAGE_DIR,
    rounds=[1],
    nodes=[1],
    variants=['test']
)

print(f"Trovate {len(filtered)} immagini")

# Crea collage
fig = create_collage(
    filtered,
    cols=4,
    show_labels=True,
    save_path="collage_node1_round1.png"
)
plt.show()

### 5. Esempio: Confronto tra varianti (test vs train vs da_embedding)

In [ ]:
# Filtra immagini: confronto tra test, train e da_embedding per una classe
filtered = filter_images(
    IMAGE_DIR,
    rounds=[1],
    nodes=[0],
    classes=['airplane'],
    variants=['test', 'train', 'da_embedding']  # Tutte le varianti
)

print(f"Trovate {len(filtered)} immagini")

# Crea collage
fig = create_collage(
    filtered,
    cols=4,
    show_labels=True,
    save_path="collage_comparison_variants.png"
)
plt.show()

### 6. Esempio personalizzato: Definisci i tuoi filtri

In [ ]:
# PERSONALIZZA QUESTI PARAMETRI
my_rounds = [1, 5, 10]  # None per tutti i round
my_nodes = [0, 1]       # None per tutti i nodi
my_classes = None       # None per tutte le classi, oppure ['airplane', 'breathing']
my_variants = ['test']  # None per tutte le varianti, oppure ['test', 'train', 'da_embedding']
my_max_images = 50      # Limite massimo di immagini

# Filtra
filtered = filter_images(
    IMAGE_DIR,
    rounds=my_rounds,
    nodes=my_nodes,
    classes=my_classes,
    variants=my_variants,
    max_images=my_max_images
)

print(f"Trovate {len(filtered)} immagini")

# Mostra alcune informazioni sulle immagini filtrate
if filtered:
    print("\nPrime 5 immagini:")
    for img in filtered[:5]:
        print(f"  - Round {img['round']}, Node {img['node']}, Class: {img['class_name']}, Variant: {img['variant']}")

# Crea collage
if filtered:
    fig = create_collage(
        filtered,
        cols=6,
        show_labels=True,
        save_path="collage_custom.png"
    )
    plt.show()

### 7. Statistiche sulle immagini filtrate

In [ ]:
from collections import Counter

def print_statistics(images_metadata: List[dict]):
    """Stampa statistiche sulle immagini filtrate."""
    if not images_metadata:
        print("Nessuna immagine da analizzare")
        return
    
    print(f"\n=== Statistiche su {len(images_metadata)} immagini ===")
    
    # Conteggi per categoria
    rounds = Counter(img['round'] for img in images_metadata)
    nodes = Counter(img['node'] for img in images_metadata)
    classes = Counter(img['class_name'] for img in images_metadata)
    variants = Counter(img['variant'] for img in images_metadata)
    
    print("\nDistribuzione per Round:")
    for round_num, count in sorted(rounds.items()):
        print(f"  Round {round_num}: {count} immagini")
    
    print("\nDistribuzione per Node:")
    for node_num, count in sorted(nodes.items()):
        print(f"  Node {node_num}: {count} immagini")
    
    print("\nDistribuzione per Classe:")
    for class_name, count in sorted(classes.items(), key=lambda x: x[1], reverse=True):
        print(f"  {class_name}: {count} immagini")
    
    print("\nDistribuzione per Variante:")
    for variant, count in sorted(variants.items()):
        print(f"  {variant}: {count} immagini")

# Esempio di utilizzo
filtered = filter_images(IMAGE_DIR, max_images=100)
print_statistics(filtered)